# LB 0.3560778

In [1]:
!pip install -r requirements.txt

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/a6/11/5171f6a1ecf7f008648fef6ef780d92414763ff5ba50a796657b9275dc1e/lightgbm-4.2.0-py3-none-manylinux_2_28_x86_64.whl.metadata
  Using cached lightgbm-4.2.0-py3-none-manylinux_2_28_x86_64.whl.metadata (19 kB)
  Obtaining dependency information for category_encoders from https://files.pythonhosted.org/packages/7f/e5/79a62e5c9c9ddbfa9ff5222240d408c1eeea4e38741a0dc8343edc7ef1ec/category_encoders-2.6.3-py2.py3-none-any.whl.metadata
  Using cached category_encoders-2.6.3-py2.py3-none-any.whl.metadata (8.0 kB)
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/c3/eb/496aa2f5d356af4185f770bc76055307f8d1870e11016b10fd779b21769c/xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata
  Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a6

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import category_encoders as ce
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import KFold
import statistics
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from scipy.spatial import distance
import time
from scipy.spatial import KDTree

In [3]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
Id = pd.DataFrame(test['Unnamed: 0'])
df.drop('Unnamed: 0',axis=1,inplace=True)
test.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
df.dtypes

created_at    object
tree_dbh       int64
curb_loc      object
health         int64
steward       object
guards        object
sidewalk      object
user_type     object
problems      object
spc_common    object
spc_latin     object
nta           object
nta_name      object
borocode       int64
boro_ct        int64
boroname      object
zip_city      object
cb_num         int64
st_senate      int64
st_assem       int64
cncldist       int64
dtype: object

In [5]:
df['nta'].nunique()

187

In [6]:
df['nta_name'].nunique()

187

In [7]:
df['health'].value_counts()

health
1    15751
0     3535
2      698
Name: count, dtype: int64

In [8]:
a = df.copy()
onehot_encoder = OneHotEncoder(sparse=False)
target_encoder = ce.TargetEncoder()
label_encoder = LabelEncoder()
binary_encoder = ce.BinaryEncoder()
Helmert_encoder = ce.HelmertEncoder()
JamesStein_encoder = ce.JamesSteinEncoder()
Backward_encoder = ce.BackwardDifferenceEncoder()
Polynomial_encoder = ce.PolynomialEncoder()
LeaveOneOut_encoder = ce.LeaveOneOutEncoder()
WOE_encoder = ce.WOEEncoder()
CatBoost_encoder = ce.CatBoostEncoder()


def apply_one_hot_encoding(df, column):
    try:
        onehot_encoded = onehot_encoder.fit_transform(df[[column]])
        for i in range(onehot_encoded.shape[1]):
            df[f'onehot_{column}_{i}'] = onehot_encoded[:, i]
    except Exception as e:
        print(f"Error in one-hot encoding for {column}: {e}")
    return df

def apply_label_encoding(df, column):
    try:
        df[f'label_{column}'] = label_encoder.fit_transform(df[column])
    except Exception as e:
        print(f"Error in label encoding for {column}: {e}")
    return df

def apply_frequency_encoding(df, column):
    try:
        frequency_encoding = df[column].value_counts(normalize=True)
        df[f'freq_{column}'] = df[column].map(frequency_encoding)
    except Exception as e:
        print(f"Error in frequency encoding for {column}: {e}")
    return df

def apply_binary_encoding(df, column):
    try:
        binary_encoded = binary_encoder.fit_transform(df[[column]])
        for i in range(binary_encoded.shape[1]):
            df[f'binary_{column}_{i}'] = binary_encoded[:, i]
    except Exception as e:
        print(f"Error in binary encoding for {column}: {e}")
    return df

def apply_helmert_encoding(df, column):
    try:
        # Assuming Helmert_encoder is previously defined and imported
        df[f'Helmert_{column}'] = Helmert_encoder.fit_transform(df[[column]])
    except Exception as e:
        print(f"Error in Helmert encoding for {column}: {e}")
    return df

def apply_backward_encoding(df, column):
    try:
        # Assuming Backward_encoder is previously defined and imported
        df[f'Backward_{column}'] = Backward_encoder.fit_transform(df[[column]])
    except Exception as e:
        print(f"Error in backward difference encoding for {column}: {e}")
    return df

def apply_polynomial_encoding(df, column):
    try:
        # Assuming Polynomial_encoder is previously defined and imported
        df[f'Polynomial_{column}'] = Polynomial_encoder.fit_transform(df[[column]])
    except Exception as e:
        print(f"Error in polynomial encoding for {column}: {e}")
    return df


def apply_encodings(df,columns):
    for column in columns:
        #df = apply_one_hot_encoding(df, column)
        df = apply_label_encoding(df, column)
        df = apply_frequency_encoding(df, column)
        #df = apply_binary_encoding(df, column)
        #df = apply_helmert_encoding(df, column)
        #df = apply_backward_encoding(df, column)
        #df = apply_polynomial_encoding(df, column)
    return df



def category(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('category')
    return df



def choose_encoding(df):
    y = df['health']
    X = df.drop('health',axis=1)
    X=category(X)
    #columns = ['curb_loc','steward','guards','sidewalk','spc_latin','nta_name','user_type','problems','boroname']
    columns = ['curb_loc','steward','guards','sidewalk','user_type','problems','spc_common','spc_latin','nta','nta_name','boroname','zip_city']
    X = apply_encodings(X,columns)
    
    X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.33,random_state=28)

    params = {
        'objective': 'multiclass',
        'num_class': 3,
        'metric': 'multi_logloss',
        #'random_state': 10, 
        'n_estimators': 1000,
        'verbose': -1,
        #'early_stopping_round': 100,
        'feature_importances':'gain'
        }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    
    feature = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
    feature.sort_values(by='importance',ascending=False)
    
    df_dict = {}
    top_elements = {}

    for col in columns:
        matched_rows = feature[feature['feature'].str.contains(col)]
        df_dict[col] = matched_rows
        
    for col in df_dict:
        sorted_df = df_dict[col].sort_values(by='importance', ascending=False)

        if not sorted_df.empty:
            top_elements[col] = sorted_df.iloc[0].to_dict()

    top_features_list = [value['feature'] for value in top_elements.values()]
    selected_features = feature[feature['feature'].isin(top_features_list)]
    return selected_features
    del y,X,df


a = choose_encoding(a)
a

,feature,importance
2,curb_loc,543
3,steward,939
4,guards,1149
5,sidewalk,1453
6,user_type,2983
10,nta,8213
31,freq_problems,4441
33,freq_spc_common,4565
34,label_spc_latin,4461
38,label_nta_name,4214


In [9]:
def category(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('category')
    return df

def date(df):
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['created_year'] = df['created_at'].dt.year
    df['created_month'] = df['created_at'].dt.month
    df['created_date'] = df['created_at'].dt.day
    df['created_weekdat'] = df['created_at'].dt.weekday
    df.drop('created_at',axis=1,inplace=True)
    return df

def encoding(df):
    label_columns = ['curb_loc','steward','guards','sidewalk','boroname','user_type']
    for col in label_columns:
        df = apply_label_encoding(df, col)
    df.drop(label_columns,axis=1,inplace=True)

    
    return df


def get_geo(df):
    locations = [
        {"name": "Little Neck, Queens", "latitude": 40.7620, "longitude": -73.73821},
        {"name": "Bronx, Bronx", "latitude": 40.837048, "longitude": -73.8654332},
        {"name": "Staten Island, Staten Island", "latitude": 40.579021, "longitude": -74.1515353},
        {"name": "New York, Manhattan", "latitude": 40.7831, "longitude": -73.97124},
        {"name": "Flushing, Queens", "latitude": 40.7658, "longitude": -73.83315},
        {"name": "Saint Albans, Queens", "latitude": 40.6948, "longitude": -73.76696},
        {"name": "Brooklyn, Brooklyn", "latitude": 40.650002, "longitude": -73.9499977},
        {"name": "Ozone Park, Queens", "latitude": 40.681938, "longitude": -73.8461468},
        {"name": "Whitestone, Queens", "latitude": 40.792045, "longitude": -73.8095579},
        {"name": "Jamaica, Queens", "latitude": 40.699783, "longitude": -73.78602610},
        {"name": "Rosedale, Queens", "latitude": 40.6584068, "longitude": -73.73895961},
        {"name": "Oakland Gardens, Queens", "latitude": 40.7408584, "longitude": -73.7582412},
        {"name": "Elmhurst, Queens", "latitude": 40.7429, "longitude": -73.88003},
        {"name": "Bellerose, Queens", "latitude": 40.7268, "longitude": -73.74154},
        {"name": "Forest Hills, Queens", "latitude": 40.719517, "longitude": -73.8522115},
        {"name": "Middle Village, Queens", "latitude": 40.7184, "longitude": -73.882296},
        {"name": "Queens Village, Queens", "latitude": 40.7156628, "longitude": -73.74190177},
        {"name": "Ridgewood, Queens", "latitude": 40.710850, "longitude": -73.897766},
        {"name": "Jackson Heights, Queens", "latitude": 40.755684, "longitude": -73.883072},
        {"name": "East Elmhurst, Queens", "latitude": 40.7737505, "longitude": -73.871309},
        {"name": "Woodside, Queens", "latitude": 40.745, "longitude": -73.9052},
        {"name": "Astoria, Queens", "latitude": 40.7710, "longitude": -73.90473},
        {"name": "Long Island City, Queens", "latitude": 40.74553, "longitude": -73.94854},
        {"name": "Cambria Heights, Queens", "latitude": 40.692158, "longitude": -73.7330755},
        {"name": "Rockaway Park, Queens", "latitude": 40.5786, "longitude": -73.84106},
        {"name": "Springfield Gardens, Queens", "latitude": 40.678159, "longitude": -73.7465217},
        {"name": "Central Park, Manhattan", "latitude": 40.785091, "longitude": -73.9682851},
        {"name": "Maspeth, Queens", "latitude": 40.7294018, "longitude": -73.90658832},
        {"name": "Howard Beach, Queens", "latitude": 40.658, "longitude": -73.840},
        {"name": "South Ozone Park, Queens", "latitude": 40.676, "longitude": -73.812},
        {"name": "Glen Oaks, Queens", "latitude": 40.7471504, "longitude": -73.71182235},
        {"name": "Bayside, Queens", "latitude": 40.758556, "longitude": -73.7654346},
        {"name": "Fresh Meadows, Queens", "latitude": 40.732689, "longitude": -73.7848667},
        {"name": "Sunnyside, Queens", "latitude": 40.743, "longitude": -73.9208},
        {"name": "Far Rockaway, Queens", "latitude": 40.6009, "longitude": -73.75701},
        {"name": "Hollis, Queens", "latitude": 40.711220, "longitude": -73.7624952},
        {"name": "Kew Gardens, Queens", "latitude": 40.705, "longitude": -73.8253},
        {"name": "Woodhaven, Queens", "latitude": 40.6901366, "longitude": -73.85660875},
        {"name": "Rego Park, Queens", "latitude": 40.7237, "longitude": -73.70496},
        {"name": "Floral Park, Queens", "latitude": 40.7239, "longitude": -73.70588},
        {"name": "Corona, Queens", "latitude": 40.735, "longitude": -73.865},
        {"name": "Richmond Hill, Queens", "latitude": 40.695, "longitude": -73.83},
        {"name": "Arverne, Queens", "latitude": 40.59, "longitude": -73.795},
        {"name": "College Point, Queens", "latitude": 40.785, "longitude": -73.835},
        {"name": "Richmond Hill, Queens (alternate)", "latitude": 40.6868, "longitude": -73.823},
        {"name": "South Richmond Hill, Queens", "latitude": 40.688364, "longitude": -73.822763}
    ]
    locations_1 = pd.DataFrame(locations)
    locations_1['combined_address']=locations_1['name']
    locations_1.drop('name',axis=1,inplace=True)

    locations_1 = locations_1.set_index('combined_address')
    
    df['combined_address'] = df['zip_city'] + ', ' + df['boroname']
    df['latitude'] = df['combined_address'].map(locations_1['latitude'])
    df['longitude'] = df['combined_address'].map(locations_1['longitude'])
    
    df.drop('combined_address',axis=1,inplace=True)
    
    return df


def tree(df):
    tree = KDTree(df[['latitude', 'longitude']])
    distances, _ = tree.query(df[['latitude', 'longitude']], k=2)

    nearest_distances = distances[:, 1]
    df['nearest_neighbor_distance'] = nearest_distances
    return df

def clustering(df):
    kmeans = KMeans(n_clusters=3, random_state=0)
    df['cluster'] = kmeans.fit_predict(df[['latitude', 'longitude']])
    return df


In [10]:
from sklearn.preprocessing import MinMaxScaler
def minmax(df):
    ms = MinMaxScaler()
    df['tree_dbh'] = ms.fit_transform(df['tree_dbh'].values.reshape(-1, 1))
    return df

In [11]:
def target_enc(col):
    #skf = StratifiedKFold(n_splits=5)
    kf = KFold(n_splits=5)
    encoded_features = []
    
    for train_idx, val_idx in kf.split(X,y):
        X_train_, X_valid_ = X.iloc[train_idx], X.iloc[val_idx]
        y_train_ = y.iloc[train_idx]

        target_encoder = ce.TargetEncoder()
        target_encoder.fit(X_train_[col], y_train_)

        X_valid_[f'target_{col}'] = target_encoder.transform(X_valid_[col])
        encoded_features.append(X_valid_)


    encoded_df = pd.concat(encoded_features).sort_index()
    df_with_encoded = pd.merge(X, encoded_df[[f'target_{col}']], left_index=True, right_index=True, how='left')

    target_encoder = ce.TargetEncoder()
    target_encoder.fit(df_with_encoded[col], y)

    test[f'target_{col}'] = target_encoder.transform(test[col])

    df_with_encoded.drop(col,axis=1,inplace=True)
    test.drop(col,axis=1,inplace=True)
    
    return df_with_encoded, test

In [12]:
def get_train(df):
    df.fillna('NULL',inplace=True)
    df = df[~df['spc_common'].isin(['Chinese chestnut', 'Himalayan cedar'])]
    df = df[~df['nta'].isin(['BK27', 'MN21', 'MN20'])]
    df = df[~df['boro_ct'].isin([3046400, 1011201, 1011203, 4112900, 3050500, 4138502, 3080200, 2005002, 4121100, 4018700, 4133900, 3011600, 3028501, 1031703, 3032100, 3044900, 2037800, 3074600, 3082800, 3091000, 4012602, 2007100, 3043900, 3035200, 3022400, 2028100, 4050500, 1008200, 3005000, 4003400, 4045900, 5018701, 3064400, 2002000, 3051602, 3077200, 4041300, 4079702, 3051100, 2039902, 2005600, 1024100, 4057700, 3025000, 3076200, 3050600, 1002602, 2026602, 2039400, 4018800, 2022000, 2034800, 3020400, 3028600, 3118201, 3007100, 2008700, 2025600, 4048000, 3028100, 3019400, 3102200, 4085900, 1013900, 4021900, 3078800, 4013200, 3091600, 3002000, 4004500, 2007700, 2028700, 1004700, 3082400, 3005601, 1012900, 3056300, 4080301, 2006700, 3098800, 3022000, 4024500, 1029300, 2040502, 2014900, 3038900, 4071100, 3021000, 4061900, 4002000, 4014800, 1006800, 4027600, 2039800, 3037401, 3114201, 3028700, 1010400, 3007200, 1023200, 3053800, 3117800, 4052200, 2038000, 4047600, 3027700, 3083000, 3019000, 3057400, 4059900, 3035900, 1004800, 2037504, 4023800, 3061002, 3026700, 2032400, 3018000, 3082000, 1012500, 1025300, 4046100, 4007700, 3051800, 3115800, 4050201, 4003100, 3030300, 4156700, 2014500, 4040700, 2022702, 2022703, 4049200, 4062000, 3102000, 2018102, 3084600, 2035000, 3037500, 3041600, 3092800, 3054400, 1006400, 3118400, 1019200, 2004800, 1001800, 2017100, 4077903, 2021200, 4018000, 3036502, 3091800, 3019100, 2033500, 3023200, 2003300, 4000100, 4055400, 2020200, 4033900, 4091900, 3065200, 3116400, 2019700, 4029300, 3030900, 5027704, 4059000, 4037500, 3039100, 4011900, 1020800, 2023301, 2023302, 1003400, 3102600, 4028300, 4045200, 3085200, 3059600, 1002900, 4049301, 3055000, 4027800, 4079000, 3012600, 3101600, 3076000, 1006500, 3020200, 1010601, 1010602, 1014700, 4009900, 3049400, 1018300, 1031100, 4026300, 2029500, 4056000, 3096000, 3023300, 2012102, 3048400, 3074000, 3039700, 1025500, 4046300, 3116000, 4012000, 1016300, 3055600, 4028400, 3123700, 2022901, 1011700, 3008500, 4011000, 4117500, 2018301])]
    df= get_geo(df)
    df = clustering(df)
    df = encoding(df)
    #df = tree(df)
    df = date(df)
    df = category(df)
    df = minmax(df)
    return df

def get_test(df):
    df.fillna('NULL',inplace=True)
    df= get_geo(df)
    df = clustering(df)
    df = encoding(df)
    #df = tree(df)
    df = date(df)
    df = category(df)
    df = minmax(df)
    return df

## 前準備

In [13]:
start = time.time()
df = get_train(df)
test = get_test(test)
end = time.time()
print(f'経過時間： {end-start}s')

経過時間： 6.403079032897949s


In [14]:
y = df['health']
X = df.drop('health',axis=1)

In [15]:
target_columns = ['spc_common','nta','zip_city','problems','spc_latin','nta_name']
for col in target_columns:
    X,test = target_enc(col)

In [16]:
X.dtypes

tree_dbh             float64
borocode               int64
boro_ct                int64
cb_num                 int64
st_senate              int64
st_assem               int64
cncldist               int64
latitude             float64
longitude            float64
cluster                int32
label_curb_loc         int64
label_steward          int64
label_guards           int64
label_sidewalk         int64
label_boroname         int64
label_user_type        int64
created_year           int32
created_month          int32
created_date           int32
created_weekdat        int32
target_spc_common    float64
target_nta           float64
target_zip_city      float64
target_problems      float64
target_spc_latin     float64
target_nta_name      float64
dtype: object

## drop_columns

In [17]:
def drop(df):
    drop_columns = ['label_boroname','borocode','target_spc_latin','target_nta_name']
    df.drop(drop_columns,axis=1,inplace=True)
    return df

In [18]:
X = drop(X)
test = drop(test)

## モデリング

In [19]:
# 標準化 (Standardization)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
"""
scaler_standard = StandardScaler()
data_standardized = scaler_standard.fit_transform(X)
data_standardized = scaler_standard.fit_transform(test)
"""
# 正規化 (Normalization)
scaler_minmax = MinMaxScaler()
data_normalized = scaler_minmax.fit_transform(X)
data_normalized = scaler_minmax.fit_transform(test)


In [20]:
from sklearn.utils import class_weight
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = dict(zip(np.unique(y), weights))

In [21]:
class_weights[0] = 1.9503831417624522
class_weights[1] = 0.4366529421856236
class_weights[2] = 7.101843547583457

In [22]:
class_weights

{0: 1.9503831417624522, 1: 0.4366529421856236, 2: 7.101843547583457}

In [23]:
from imblearn.under_sampling import RandomUnderSampler
sampling_strategy = {1: int(15545*0.5), 0: int(3481*0.5),2:669} # 0:多数派クラス, 1:少数派クラス

# アンダーサンプリングの設定と実行
under_sampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=28)
X_resampled, y_resampled = under_sampler.fit_resample(X, y)

# 結果の表示
print("元のデータセットのラベルのカウント:", dict(zip(*np.unique(y, return_counts=True))))
print("リサンプリング後のラベルのカウント:", dict(zip(*np.unique(y_resampled, return_counts=True))))

元のデータセットのラベルのカウント: {0: 3481, 1: 15545, 2: 669}
リサンプリング後のラベルのカウント: {0: 1740, 1: 7772, 2: 669}


In [24]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import statistics
from scipy.stats import mode

average_f1 = []
predictions = []
skf = StratifiedKFold(n_splits=5)
for train_idx, val_idx in skf.split(X_resampled,y_resampled):
    X_train, X_val = X_resampled.iloc[train_idx], X_resampled.iloc[val_idx]
    y_train, y_val = y_resampled.iloc[train_idx], y_resampled.iloc[val_idx]
    
    params = {
        'objective': 'multiclass',
        'num_class': 3,
        'metric': 'multi_logloss',
        'class_weight': class_weights,
        'n_estimators': 1000,
        'verbose': -1,
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mean_f1 = f1_score(y_val, y_pred, average='macro')
    average_f1.append(mean_f1)
    feature_gains = model.booster_.feature_importance(importance_type='gain')
    feature = pd.DataFrame({'feature': X.columns, 'importance': feature_gains})
    feature.sort_values(by='importance',ascending=False)
    print(feature)
    print("Mean F1 Score (Macro F1 Score):", mean_f1)
    
    predictions.append(model.predict(test))

average_f1 = statistics.mean(average_f1)
print(' ')
print(f'average_f1:{average_f1}')

weighted_votes = np.zeros((len(test), len(class_weights)))

for prediction in predictions:
    for idx, pred in enumerate(prediction):
        weighted_votes[idx, pred] += class_weights[pred]

weighted_majority_vote = np.argmax(weighted_votes, axis=1)

skf_submission = pd.DataFrame({'Id': Id['Unnamed: 0'], 'predict': weighted_majority_vote})


              feature    importance
0            tree_dbh   8736.716187
1             boro_ct   8162.519876
2              cb_num   2964.117211
3           st_senate   2734.054064
4            st_assem   4761.254468
5            cncldist   3238.192980
6            latitude   1546.599710
7           longitude   1803.674899
8             cluster    289.356551
9      label_curb_loc    420.292856
10      label_steward   2205.563905
11       label_guards   1303.034530
12     label_sidewalk   1263.090472
13    label_user_type   1774.190362
14       created_year    893.979351
15      created_month   5078.418664
16       created_date   9666.762337
17    created_weekdat   5209.712086
18  target_spc_common  13108.693729
19         target_nta  11901.313753
20    target_zip_city   6871.624630
21    target_problems   8441.655358
Mean F1 Score (Macro F1 Score): 0.34090609635589614
              feature    importance
0            tree_dbh   8818.287774
1             boro_ct   8991.119088
2           

In [25]:
#skf_submission.to_csv('submission_1-2_2122.csv', index=False, header=False)

In [26]:
skf_submission['predict'].value_counts()

predict
1    15206
0     3839
2      657
Name: count, dtype: int64

In [27]:
import numpy as np
from sklearn.metrics import f1_score

# LightGBMモデルを訓練し、検証データセット上での予測確率を取得
# model = 訓練済みのLightGBMモデル
# X_val = 検証データセットの特徴量
predicted_probabilities = model.predict(X_val)

# 閾値を変えながらF1スコアを計算
best_threshold = 0
best_score = 0

for threshold in np.linspace(0, 1, 100):
    # 閾値を用いて確率をクラスラベルに変換
    predicted_labels = (predicted_probabilities > threshold).astype(int)

    # macro F1スコアを計算
    score = f1_score(y_val, predicted_labels, average='macro')

    if score > best_score:
        best_score = score
        best_threshold = threshold

print(f"Best Threshold: {best_threshold}, Best macro F1 Score: {best_score}")


Best Threshold: 0.0, Best macro F1 Score: 0.3237089067572702
